In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from mne import Epochs, pick_types, events_from_annotations
from mne.channels import make_standard_montage
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP
import random

In [ ]:
tmin, tmax = -1.0, 4.0
event_id = dict(rest=1,left=2, right=3)
runs = [3]

train_data = open("PhysionetTrain.csv", "a")
test_data = open("PhysionetTest.csv", "a")
val_data = open("PhysionetVal.csv", "a")

train_data.write('paciente;'+'medicao;'+'eletrodo;'+'label')
val_data.write('paciente;'+'medicao;'+'eletrodo;'+'label')
test_data.write('paciente;'+'medicao;'+'eletrodo;'+'label')


for idx in range(640):
    train_data.write(';leitura'+str(idx))
    val_data.write(';leitura'+str(idx))
    test_data.write(';leitura'+str(idx))

train_data.write('\n')
val_data.write('\n')
test_data.write('\n')

teste = open("teste2.csv", "a")
for subject in range(1,110):
    raw_fnames = eegbci.load_data(subject, runs)
    raw = concatenate_raws([read_raw_edf(f, preload=True) for f in raw_fnames])
    eegbci.standardize(raw)  # set channel names
    montage = make_standard_montage("standard_1005")
    raw.set_montage(montage)
    raw.filter(7.0, 30.0, fir_design="firwin", skip_by_annotation="edge")
    events, _ = events_from_annotations(raw, event_id='auto')
    picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False, exclude="bads")
    channels = raw.ch_names
    channels = [x.replace('.','').upper() for x in channels]
    epochs = Epochs(
        raw,
        events,
        event_id,
        tmin,
        tmax,
        proj=True,
        picks=picks,
        baseline=None,
        preload=True,
    )
    epochs_data = epochs.get_data()
    labels = epochs.events[:, -1]
    flag = True
    random_epochs = random.sample(range(0, 29), 29)
    epochs_val = random_epochs[0:4]
    epochs_test = random_epochs[4:8]
    for epoch in range(len(epochs_data)):
        label = labels[epoch]
        if(label == 1):
            if(flag == True):
                flag = False
            else:
                flag = True
        
        if(label != 1 or (label == 1 and flag)):
            if(epoch in epochs_val):
                for eletrode in range(len(epochs_data[epoch])):
                    val_data.write(str(subject)+';'+str(epoch+1)+';'+channels[eletrode]+';'+str(label))
                    sign = epochs_data[epoch][eletrode][:640]
                    
                    for leitura in range(len(sign)):
                        val_data.write(';'+str(sign[leitura]))
                    val_data.write('\n')
            elif(epoch in epochs_test):
                for eletrode in range(len(epochs_data[epoch])):
                    test_data.write(str(subject)+';'+str(epoch+1)+';'+channels[eletrode]+';'+str(label))
                    sign = epochs_data[epoch][eletrode][:640]
                    
                    for leitura in range(len(sign)):
                        test_data.write(';'+str(sign[leitura]))
                    test_data.write('\n')
            else:
                for eletrode in range(len(epochs_data[epoch])):
                    train_data.write(str(subject)+';'+str(epoch+1)+';'+channels[eletrode]+';'+str(label))
                    sign = epochs_data[epoch][eletrode][:640]
                    
                    for leitura in range(len(sign)):
                        train_data.write(';'+str(sign[leitura]))
                    train_data.write('\n')
train_data.close()
val_data.close()
test_data.close()
        


In [91]:
train_dataset = pd.read_csv('PhysionetTrain.csv', delimiter=';')
test_dataset = pd.read_csv('PhysionetTest.csv', delimiter=';')
val_dataset = pd.read_csv('PhysionetVal.csv', delimiter=';')

In [92]:
print(train_dataset['label'].value_counts()[1],train_dataset['label'].value_counts()[2],train_dataset['label'].value_counts()[3])
print(val_dataset['label'].value_counts()[1],val_dataset['label'].value_counts()[2],val_dataset['label'].value_counts()[3])
print(test_dataset['label'].value_counts()[1],test_dataset['label'].value_counts()[2],test_dataset['label'].value_counts()[3])

36736 38720 38144
6080 6528 7872
6336 7040 6464
